In [1]:
import pandas as pd
import numpy as np

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold,cross_val_score
from lightgbm import LGBMClassifier
#from xgboost import XGBClassifier
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()
sns.set_style('darkgrid')
import warnings
warnings.filterwarnings('ignore')
#from bayes_opt import BayesianOptimization
from sklearn.ensemble import (RandomForestClassifier, ExtraTreesClassifier)
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import gc
gc.enable()

In [2]:
print('Read data and test')
data = pd.read_csv('./input/application_train.csv')
test = pd.read_csv('./input/application_test.csv')
print('Shapes : ', data.shape, test.shape)
y = data['TARGET']
ID = test['SK_ID_CURR']
del data,test
gc.collect()

Read data and test
Shapes :  (307511, 122) (48744, 121)


35

In [3]:
rf_oof_train = pd.read_csv('./output/rf_oof_train.csv')
rf_oof_test = pd.read_csv('./output/rf_oof_test.csv')
et_oof_train = pd.read_csv('./output/et_oof_train.csv')
et_oof_test = pd.read_csv('./output/et_oof_test.csv')
xgb_oof_train2= pd.read_csv('./output/xgb_oof_train2.csv')
xgb_oof_test3 = pd.read_csv('./output/xgb_oof_test3.csv')
xgb_oof_train3= pd.read_csv('./output/xgb_oof_train3.csv')
xgb_oof_test2 = pd.read_csv('./output/xgb_oof_test2.csv')
xgb_oof_train = pd.read_csv('./output/xgb_oof_train.csv')
xgb_oof_test = pd.read_csv('./output/xgb_oof_test.csv')
xgb_oof_train1 = pd.read_csv('./output/xgb_oof_train1.csv')
xgb_oof_test1 = pd.read_csv('./output/xgb_oof_test1.csv')
log_oof_train = pd.read_csv('./output/log_oof_train.csv')
log_oof_test = pd.read_csv('./output/log_oof_test.csv')
lgb_oof_train = pd.read_csv('./output/lgb_oof_train.csv')
lgb_oof_test = pd.read_csv('./output/lgb_oof_test.csv')
lgb_oof_train2 = pd.read_csv('./output/lgb_oof_train2.csv')
lgb_oof_test2 = pd.read_csv('./output/lgb_oof_test2.csv')
lgb_oof_train4 = pd.read_csv('./output/lgb_oof_train3.csv')
lgb_oof_test4 = pd.read_csv('./output/lgb_oof_test3.csv')
lgb_oof_train3 = pd.read_csv('./output/lightGBM_out_of_fold_train_predictions.csv')
lgb_oof_test3 = pd.read_csv('./output/lightGBM_out_of_fold_test_predictions.csv')
lgb_oof_train5 = pd.read_csv('./output/lightGBM_out_of_fold_train_predictions1.csv')
lgb_oof_test5 = pd.read_csv('./output/lightGBM_out_of_fold_test_predictions1.csv')
jeepy_oof_train = pd.read_csv('./output/jeepy_oof_train.csv')
jeepy_oof_test = pd.read_csv('./output/jeepy_oof_test.csv')
nn_oof_train = pd.read_csv('./output/nn_oof_train.csv')
nn_oof_test = pd.read_csv('./output/nn_oof_test.csv')
WEIGHT_AVERAGE_RANK = pd.read_csv('WEIGHT_AVERAGE_RANK.csv')
cv_blending3 = pd.read_csv('cv_blending3.csv')
cv7994 = pd.read_csv('stacking-cv-7994.csv')
cv7997 = pd.read_csv('cv-7997.csv')
hope = pd.read_csv('hope.csv')
cat_oof_train = pd.read_csv('./output/catboost_out_of_fold_train_predictions.csv')
cat_oof_test = pd.read_csv('./output/catboost_out_of_fold_test_predictions.csv')

In [20]:
data = pd.DataFrame()
data['SK_ID_CURR']=rf_oof_train['ID']
data['rf_oof']=rf_oof_train['rf_oof_train']
data['log_oof'] =log_oof_train['log_oof_train']
data['et_oof'] =et_oof_train['et_oof_train']
#data['xgb_oof2']=xgb_oof_train2['xgb_oof_train']
data['xgb_oof1']=xgb_oof_train1['xgb_oof_train']
data['xgb_oof']=xgb_oof_train['xgb_oof_train']
data['lgb_oof2']=lgb_oof_train2['lgb_oof_train']
data['lgb_oof4']=lgb_oof_train4['lgb_oof_train']
data=data.merge(lgb_oof_train3[['SK_ID_CURR','lightGBM_prediction']],on='SK_ID_CURR',how='left')
data=data.merge(lgb_oof_train5[['SK_ID_CURR','lightGBM_prediction']],on='SK_ID_CURR',how='left')
data['lgb_oof']=lgb_oof_train['lgb_oof_train']
data['jeepy_oof']=jeepy_oof_train['jeepy_oof_train']
data=data.merge(cat_oof_train[['SK_ID_CURR','catboost_prediction']],on='SK_ID_CURR',how='left')
data['nn_oof'] = nn_oof_train['nn_oof_train']

In [6]:
data.corr()

,SK_ID_CURR,log_oof,et_oof,xgb_oof1,xgb_oof,lgb_oof2,lgb_oof4,lightGBM_prediction_x,lightGBM_prediction_y,lgb_oof,jeepy_oof,catboost_prediction
SK_ID_CURR,1.000000,0.003959,0.000268,-0.002113,-0.002435,-0.002461,-0.002202,-0.002167,-0.001616,-0.002525,-0.003281,-0.002764
log_oof,0.003959,1.000000,0.535238,0.278652,0.280225,0.266206,0.264152,0.266791,0.266159,0.265846,0.271590,0.272251
et_oof,0.000268,0.535238,1.000000,0.587025,0.590485,0.551448,0.547204,0.549358,0.550679,0.551380,0.563564,0.553158
xgb_oof1,-0.002113,0.278652,0.587025,1.000000,0.987754,0.966239,0.950940,0.923653,0.920291,0.956389,0.904810,0.904283
xgb_oof,-0.002435,0.280225,0.590485,0.987754,1.000000,0.958286,0.959007,0.924290,0.921749,0.964739,0.903967,0.903156
lgb_oof2,-0.002461,0.266206,0.551448,0.966239,0.958286,1.000000,0.975862,0.932551,0.931456,0.981918,0.907505,0.909989
lgb_oof4,-0.002202,0.264152,0.547204,0.950940,0.959007,0.975862,1.000000,0.935736,0.936304,0.989737,0.899550,0.911643
lightGBM_prediction_x,-0.002167,0.266791,0.549358,0.923653,0.924290,0.932551,0.935736,1.000000,0.965185,0.933160,0.890516,0.935818
lightGBM_prediction_y,-0.001616,0.266159,0.550679,0.920291,0.921749,0.931456,0.936304,0.965185,1.000000,0.932511,0.887361,0.957485
lgb_oof,-0.002525,0.265846,0.551380,0.956389,0.964739,0.981918,0.989737,0.933160,0.932511,1.000000,0.904919,0.907723


In [21]:
test=pd.DataFrame()
test['SK_ID_CURR']=rf_oof_test['ID']
test['rf_oof']=rf_oof_test['rf_oof_test']
test['log_oof'] =log_oof_test['log_oof_test']
test['et_oof'] =et_oof_test['et_oof_test']
#test['xgb_oof2']=xgb_oof_test2['xgb_oof_test']
test['xgb_oof1']=xgb_oof_test1['xgb_oof_test']
test['xgb_oof']=xgb_oof_test['xgb_oof_test']
test['lgb_oof2']=lgb_oof_test2['lgb_oof_test']
test['lgb_oof4']=lgb_oof_test4['lgb_oof_test']
test['lightGBM_prediction_x']=lgb_oof_test3['lightGBM_prediction']
test['lightGBM_prediction_y']=lgb_oof_test5['lightGBM_prediction']
test['lgb_oof']=lgb_oof_test['lgb_oof_test'] 
test['jeepy_oof']=jeepy_oof_test['jeepy_oof_test']
test['catboost_prediction']=cat_oof_test['catboost_prediction']
test['nn_oof'] = nn_oof_test['nn_oof_test']

In [9]:
test.corr()

,SK_ID_CURR,rf_oof,log_oof,et_oof,xgb_oof1,xgb_oof,lgb_oof2,lgb_oof4,lightGBM_prediction_x,lightGBM_prediction_y,lgb_oof,jeepy_oof,catboost_prediction
SK_ID_CURR,1.000000,-0.004862,0.000273,-0.000241,-0.002525,-0.002393,-0.003303,-0.003903,-0.002444,-0.002891,-0.002951,-0.005172,-0.002228
rf_oof,-0.004862,1.000000,0.417734,0.768712,0.789202,0.790716,0.764490,0.756224,0.749374,0.756615,0.760640,0.740642,0.755712
log_oof,0.000273,0.417734,1.000000,0.476745,0.234507,0.233786,0.228577,0.224938,0.233158,0.234935,0.226321,0.244093,0.236859
et_oof,-0.000241,0.768712,0.476745,1.000000,0.565494,0.564737,0.533201,0.525619,0.522422,0.526322,0.528645,0.548017,0.521944
xgb_oof1,-0.002525,0.789202,0.234507,0.565494,1.000000,0.990767,0.975087,0.957694,0.928580,0.924125,0.963934,0.906708,0.903174
xgb_oof,-0.002393,0.790716,0.233786,0.564737,0.990767,1.000000,0.968170,0.966622,0.930501,0.926724,0.973282,0.906920,0.901975
lgb_oof2,-0.003303,0.764490,0.228577,0.533201,0.975087,0.968170,1.000000,0.980739,0.935936,0.934057,0.986808,0.907226,0.905559
lgb_oof4,-0.003903,0.756224,0.224938,0.525619,0.957694,0.966622,0.980739,1.000000,0.938209,0.939481,0.992287,0.898619,0.906271
lightGBM_prediction_x,-0.002444,0.749374,0.233158,0.522422,0.928580,0.930501,0.935936,0.938209,1.000000,0.960780,0.936600,0.888391,0.925776
lightGBM_prediction_y,-0.002891,0.756615,0.234935,0.526322,0.924125,0.926724,0.934057,0.939481,0.960780,1.000000,0.934999,0.882710,0.949820


In [8]:
roc_auc_score(y, lgb_oof_train3['lightGBM_prediction'])

0.49902428137948923

In [6]:
submission = pd.DataFrame()
submission['SK_ID_CURR'] = ID
submission['TARGET'] = cv7997.TARGET*0.25+cv_blending3.TARGET*0.75
submission.to_csv('cv_blending24.csv', index=False)

In [26]:
submission = pd.DataFrame()
submission['SK_ID_CURR'] = ID
submission['TARGET'] = sub_preds
submission.to_csv('cv-7997.csv', index=False)

In [24]:
folds = KFold(n_splits=5, shuffle=True, random_state=1700)
sub_preds = np.zeros(test.shape[0])
feats_trn = [f for f in data.columns if f != 'SK_ID_CURR']
feats_test = [f for f in test.columns if f != 'SK_ID_CURR']
val_auc = None

In [25]:
for n_fold, (trn_idx, val_idx) in enumerate(folds.split(data)):
    trn_x, trn_y = data[feats_trn].iloc[trn_idx], y.iloc[trn_idx]
    val_x, val_y = data[feats_trn].iloc[val_idx], y.iloc[val_idx] 
    clf = LGBMClassifier(
        n_estimators=10000,
        learning_rate=0.01,
        num_leaves=30,
        colsample_bytree=0.5,
        subsample=0.1,
        max_depth=6,
        reg_alpha=1,
        reg_lambda=1,
        min_split_gain=0.1,
        min_child_weight=30,
        random_state=1700,
    )
    clf.fit(trn_x, trn_y, 
            eval_set= [(trn_x, trn_y), (val_x, val_y)], 
            eval_metric='auc', verbose=100, early_stopping_rounds=300  #30
           )
    oof_preds = clf.predict_proba(val_x)[:, 1]
    sub_preds += clf.predict_proba(test[feats_test])[:, 1] / folds.n_splits
    print('Fold %2d AUC : %.6f' % (n_fold + 1, roc_auc_score(val_y, oof_preds)))
    if n_fold == 0:
        val_auc = roc_auc_score(val_y, oof_preds)
    else:
        val_auc += roc_auc_score(val_y, oof_preds)
    del trn_x, trn_y, val_x, val_y
    gc.collect()
    
print('Full AUC score %.6f' % (val_auc/folds.n_splits) )

Training until validation scores don't improve for 300 rounds.
[100]	training's auc: 0.802388	valid_1's auc: 0.800204
[200]	training's auc: 0.803723	valid_1's auc: 0.800418
[300]	training's auc: 0.805125	valid_1's auc: 0.800537
[400]	training's auc: 0.806514	valid_1's auc: 0.800548
[500]	training's auc: 0.807664	valid_1's auc: 0.800501
[600]	training's auc: 0.808333	valid_1's auc: 0.800446
Early stopping, best iteration is:
[329]	training's auc: 0.80553	valid_1's auc: 0.80059
Fold  1 AUC : 0.800590
Training until validation scores don't improve for 300 rounds.
[100]	training's auc: 0.80155	valid_1's auc: 0.803941
[200]	training's auc: 0.802908	valid_1's auc: 0.804013
[300]	training's auc: 0.804373	valid_1's auc: 0.804104
[400]	training's auc: 0.805851	valid_1's auc: 0.804167
[500]	training's auc: 0.807024	valid_1's auc: 0.804093
[600]	training's auc: 0.807761	valid_1's auc: 0.804048
Early stopping, best iteration is:
[346]	training's auc: 0.805041	valid_1's auc: 0.804187
Fold  2 AUC : 

In [22]:
folds = KFold(n_splits=5, shuffle=True, random_state=1700)
sub_preds = np.zeros(test.shape[0])
feats_trn = [f for f in data.columns if f != 'SK_ID_CURR']
feats_test = [f for f in test.columns if f != 'SK_ID_CURR']
val_auc = None

In [23]:
for n_fold, (trn_idx, val_idx) in enumerate(folds.split(data)):
    trn_x, trn_y = data[feats_trn].iloc[trn_idx], y.iloc[trn_idx]
    val_x, val_y = data[feats_trn].iloc[val_idx], y.iloc[val_idx] 
    clf = LGBMClassifier(
        n_estimators=10000,
        learning_rate=0.01,
        num_leaves=30,
        colsample_bytree=0.5,
        subsample=0.1,
        max_depth=8,
        reg_alpha=1,
        reg_lambda=1,
        min_split_gain=0.1,
        min_child_weight=30,
        random_state=1700,
    )
    clf.fit(trn_x, trn_y, 
            eval_set= [(trn_x, trn_y), (val_x, val_y)], 
            eval_metric='auc', verbose=100, early_stopping_rounds=300  #30
           )
    oof_preds = clf.predict_proba(val_x)[:, 1]
    sub_preds += clf.predict_proba(test[feats_test])[:, 1] / folds.n_splits
    print('Fold %2d AUC : %.6f' % (n_fold + 1, roc_auc_score(val_y, oof_preds)))
    if n_fold == 0:
        val_auc = roc_auc_score(val_y, oof_preds)
    else:
        val_auc += roc_auc_score(val_y, oof_preds)
    del trn_x, trn_y, val_x, val_y
    gc.collect()
    
print('Full AUC score %.6f' % (val_auc/folds.n_splits) )

Training until validation scores don't improve for 300 rounds.
[100]	training's auc: 0.802597	valid_1's auc: 0.800274
[200]	training's auc: 0.804115	valid_1's auc: 0.800474
[300]	training's auc: 0.805705	valid_1's auc: 0.800665
[400]	training's auc: 0.807331	valid_1's auc: 0.800687
[500]	training's auc: 0.808838	valid_1's auc: 0.80065
[600]	training's auc: 0.809892	valid_1's auc: 0.800612
Early stopping, best iteration is:
[367]	training's auc: 0.806764	valid_1's auc: 0.80073
Fold  1 AUC : 0.800730
Training until validation scores don't improve for 300 rounds.
[100]	training's auc: 0.801653	valid_1's auc: 0.804028
[200]	training's auc: 0.803146	valid_1's auc: 0.804152
[300]	training's auc: 0.804803	valid_1's auc: 0.804257
[400]	training's auc: 0.806542	valid_1's auc: 0.804262
[500]	training's auc: 0.808189	valid_1's auc: 0.804104
[600]	training's auc: 0.809419	valid_1's auc: 0.803978
Early stopping, best iteration is:
[371]	training's auc: 0.806026	valid_1's auc: 0.804287
Fold  2 AUC :